In [6]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [24]:
df = pd.read_csv("../data/adult.csv").replace("?", np.nan).dropna()
df.head()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25


# Task 1
Print the count of men and women in the dataset.

In [21]:
df["sex"].value_counts()

Male      20380
Female     9782
Name: sex, dtype: int64

# Task 2
Find the average age of men in dataset

In [25]:
df[df["sex"] == "Male"]["age"].mean()

39.18400392541707

# Task 3
Get the percentage of people from Poland (native-country)

In [28]:
df[df["native-country"] == "Poland"]["native-country"].count() / df["native-country"].count()

0.0018566408063125787

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [85]:
higher = df[df["salary"] == ">50K"]["age"].mean(), df[df["salary"] == ">50K"]["age"].std()
lower = df[df["salary"] == "<=50K"]["age"].mean(), df[df["salary"] == "<=50K"]["age"].std()

In [86]:
print(higher)

(43.95911028236548, 10.269632835673852)


In [87]:
print(lower)

(36.60806038668668, 13.464631257161633)


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [50]:
df_without_higher = df[~df["education"].isin(["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"])]
len(df_without_higher[df_without_higher["salary"] == ">50K"])

3178

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [54]:
df.groupby(by="education")["age"].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [82]:
men = df.query('sex=="Male"')[["marital-status", "salary"]]
married_status = ["Married-civ-spouse", "Married-spouse-absent", "Married-AF-spouse"]
not_married_status = ["Never-married", "Divorced", "Separated", "Widowed"]
married = men[men["marital-status"].isin(married_status)].groupby("salary").value_counts()
not_married = men[men["marital-status"].isin(not_married_status)].groupby("salary").value_counts()

In [84]:
married

salary  marital-status       
<=50K   Married-civ-spouse       6886
        Married-spouse-absent     161
        Married-AF-spouse           5
>50K    Married-civ-spouse       5699
        Married-spouse-absent      20
        Married-AF-spouse           4
dtype: int64

In [83]:
not_married

salary  marital-status
<=50K   Never-married     5107
        Divorced          1407
        Separated          316
        Widowed            102
>50K    Never-married      307
        Divorced           278
        Separated           49
        Widowed             39
dtype: int64

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [88]:
max_hours_per_week = df["hours-per-week"].max()
max_hours_per_week

99

In [89]:
len(df[df["hours-per-week"] == max_hours_per_week])

78

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [90]:
df["married"] = df["marital-status"].str.startswith("Married")
df["higher_education"] = df["education"].isin(["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"])
df.groupby("sex").corr()["salary K$"]

sex                     
Female  Unnamed: 0         -0.005497
        age                 0.123477
        hours-per-week      0.113233
        salary K$           1.000000
        married             0.399906
        higher_education    0.205569
Male    Unnamed: 0          0.002299
        age                 0.227255
        hours-per-week      0.178803
        salary K$           1.000000
        married             0.318457
        higher_education    0.274518
Name: salary K$, dtype: float64

In [91]:
df.groupby("native-country")[["salary K$", "married", "higher_education"]].corr()["salary K$"]

native-country                  
Cambodia        salary K$           1.000000
                married             0.443513
                higher_education    0.323582
Canada          salary K$           1.000000
                married             0.536483
                                      ...   
Vietnam         married             0.251304
                higher_education    0.102208
Yugoslavia      salary K$           1.000000
                married             0.460488
                higher_education    0.173711
Name: salary K$, Length: 123, dtype: float64

In [93]:
df_race_salary = df[["race", "salary K$"]]
df_race_salary = pd.get_dummies(df_race_salary, columns=["race"])
df_race_salary.corr()["salary K$"]

salary K$                  1.000000
race_Amer-Indian-Eskimo   -0.026346
race_Asian-Pac-Islander    0.011248
race_Black                -0.074815
race_Other                -0.028320
race_White                 0.071669
Name: salary K$, dtype: float64